In [3]:
from time import sleep
from typing import Optional, List
from bs4 import BeautifulSoup, Tag
import requests

def get_realtime_price(url: str = 'https://finance.naver.com/sise/sise_index.naver?code=KOSDAQ') :
    req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    dom = BeautifulSoup(req.text, "html.parser")
    cur_price = dom.select_one('em#now_value').text
    cur_volume = dom.select_one('td#quant').text
    cur_volume_price = dom.select_one('td#amount').text
    cur_volume_personal = dom.select_one('dl.lst_kos_info').select_one(':nth-child(2)').select_one('span').text
    cur_volume_forg = dom.select_one('dl.lst_kos_info').select_one(':nth-child(3)').select_one('span').text
    cur_volume_instfund = dom.select_one('dl.lst_kos_info').select_one(':nth-child(4)').select_one('span').text

    cur_volume_prog_pos = dom.select_one('dl.lst_kos_info').select_one(':nth-child(6)').select_one('span').text
    cur_volume_prog_neg = dom.select_one('dl.lst_kos_info').select_one(':nth-child(7)').select_one('span').text
    
    print(cur_volume_personal + '|' + cur_volume_forg + '|' + cur_volume_instfund + '|' 
          + cur_volume_prog_pos + '|' + cur_volume_prog_neg)
    print(cur_price + "|" + cur_volume + "|" + cur_volume_price + "|" )
    

get_realtime_price()
    

+3,119억|-2,592억|-443억|+76억|-2,455억
778.15|780,027|5,696,591|


In [4]:
# 코스닥 종가
from typing import Optional, List
from bs4 import BeautifulSoup, Tag
import requests
from dataclasses import dataclass
from utils.data_type_converter import convert_number

@dataclass
class RealtimePriceCrawl:
    date: str
    closing_price: float
    up_down_per: float
    volume: float
    volume_amt: float

def get_kosdaq_daily_price(page = 1) :
    url = 'https://finance.naver.com/sise/sise_index_day.naver?code=KOSDAQ&page=%s' % page
    req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    dom = BeautifulSoup(req.text, "html.parser")
    
    v_date = dom.select('tr > td.date')
    
    res: List[RealtimePriceCrawl] = []
    for v in v_date:
        # print('---------------------------------------')
        # print(v.parent.select_one(':nth-child(1)').text)
        # print(v.parent.select_one(':nth-child(2)').text)
        # print(v.parent.select_one(':nth-child(4)').text.strip())
        # print(v.parent.select_one(':nth-child(5)').text)
        # print(v.parent.select_one(':nth-child(6)').text)
        res.append(RealtimePriceCrawl(
            date=v.parent.select_one(':nth-child(1)').text, 
            closing_price=convert_number(v.parent.select_one(':nth-child(2)').text),
            up_down_per=convert_number(v.parent.select_one(':nth-child(4)').text.strip()),
            volume=convert_number(v.parent.select_one(':nth-child(5)').text),
            volume_amt=convert_number(v.parent.select_one(':nth-child(6)').text)
            )
        )
    return res
    
get_kosdaq_daily_price()

[RealtimePriceCrawl(date='2024.08.21', closing_price=778.15, up_down_per=-1.18, volume=780027.0, volume_amt=5696591.0),
 RealtimePriceCrawl(date='2024.08.20', closing_price=787.44, up_down_per=1.28, volume=997056.0, volume_amt=7990594.0),
 RealtimePriceCrawl(date='2024.08.19', closing_price=777.47, up_down_per=-1.13, volume=1135628.0, volume_amt=7653033.0),
 RealtimePriceCrawl(date='2024.08.16', closing_price=786.33, up_down_per=1.22, volume=1080402.0, volume_amt=8000416.0),
 RealtimePriceCrawl(date='2024.08.14', closing_price=776.83, up_down_per=1.56, volume=1111020.0, volume_amt=7691492.0),
 RealtimePriceCrawl(date='2024.08.13', closing_price=764.86, up_down_per=-1.02, volume=1170514.0, volume_amt=7661966.0)]

In [5]:
import pymysql

def save_data(crawled: List[RealtimePriceCrawl] = [], type_code = 'KOSDAQ'):
    conn = pymysql.connect(host='localhost', user='root', password='root', db='zio', charset='utf8mb4')
    cur = conn.cursor()
    
    sql = f"""
        INSERT INTO CRAWL_MARKET_DAILY_DATA(TYPE_CODE, DATEON, CLOSING_PRICE, UP_DOWN_PER, VOLUME, VOLUME_AMT)
        VALUES(%s, %s, %s, %s, %s, %s) as new
        ON DUPLICATE KEY UPDATE 
            TYPE_CODE=new.TYPE_CODE,
            DATEON=new.DATEON,
            CLOSING_PRICE=new.CLOSING_PRICE,
            UP_DOWN_PER=new.UP_DOWN_PER,
            VOLUME=new.VOLUME,
            VOLUME_AMT=new.VOLUME_AMT
    """
    
    for row in crawled:
        try:
            cur.execute(sql, (type_code, row.date, row.closing_price, row.up_down_per, row.volume, row.volume_amt))
        except Exception as e :
            print('duplicated key error --> ' + str(e))
    
    conn.commit()
    conn.close()



In [8]:
crawled = get_kosdaq_daily_price(2)
print('crawled : %s' % len(crawled))
save_data(crawled, 'KOSDAQ')



crawled : 6


In [13]:
# from 2 to 1176
import time

for i in range(721, 1176):
    print(i, '/1176')
    save_data(get_kosdaq_daily_price(i), type_code = 'KOSDAQ')
    time.sleep(1)


721 /1176
722 /1176
723 /1176
724 /1176
725 /1176
726 /1176
727 /1176
728 /1176
729 /1176
730 /1176
731 /1176
732 /1176
733 /1176
734 /1176
735 /1176
736 /1176
737 /1176
738 /1176
739 /1176
740 /1176
741 /1176
742 /1176
743 /1176
744 /1176
745 /1176
746 /1176
747 /1176
748 /1176
749 /1176
750 /1176
751 /1176
752 /1176
753 /1176
754 /1176
755 /1176
756 /1176
757 /1176
758 /1176
759 /1176
760 /1176
761 /1176
762 /1176
763 /1176
764 /1176
765 /1176
766 /1176
767 /1176
768 /1176
769 /1176
770 /1176
771 /1176
772 /1176
773 /1176
774 /1176
775 /1176
776 /1176
777 /1176
778 /1176
779 /1176
780 /1176
781 /1176
782 /1176
783 /1176
784 /1176
785 /1176
786 /1176
787 /1176
788 /1176
789 /1176
790 /1176
791 /1176
792 /1176
793 /1176
794 /1176
795 /1176
796 /1176
797 /1176
798 /1176
799 /1176
800 /1176
801 /1176
802 /1176
803 /1176
804 /1176
805 /1176
806 /1176
807 /1176
808 /1176
809 /1176
810 /1176
811 /1176
812 /1176
813 /1176
814 /1176
815 /1176
816 /1176
817 /1176
818 /1176
819 /1176
820 /1176


In [1]:
%load_ext autoreload
%autoreload 2

from utils.data_type_converter import DataConverter

a = DataConverter()
a.convert_number('-20.1%')
# 
# convert_number('-20.1%')
# a = utils.data_type_converter.DataConverter()
# a.convert_number('-20.2')

-20.1

In [5]:
%reload_ext autoreload

import utils
print(utils.VERSION)

3.5


In [6]:
import utils

Initializing utils ...
